In [ ]:
%matplotlib inline 

In [ ]:
%run weatherDiagramCreator.py

In [1]:
%matplotlib inline
# %matplotlib

import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import yaml
import pandas as pd
from datetime import datetime, timedelta
import os
import re
import pprint

# location, areaname, event, time
eventStartPattern = re.compile(r'Started event ([A-Za-z0-9_]+) ([A-Za-z0-9_]+) ([A-Za-z0-9_]+) at ([T0-9-:]+)')
# location, areaname, event, time
eventEndPattern = re.compile(r'Ended event ([A-Za-z0-9_]+) ([A-Za-z0-9_]+) ([A-Za-z0-9_]+) at ([T0-9-:]+)')
eventTablePattern = re.compile(r"([a-zA-Z_0-9]+) +([a-zA-Z_0-9]+) +([a-zA-Z_0-9]+) +([T0-9:-]+) +- +([T0-9:-]+)")
thresholdPattern = re.compile(r".+_val=([A-Za-z0-9.-]+), .+_threshold=([A-Za-z0-9.-]+)")

def aggregateDiagram_rain_noNaN():
    plt.rcParams.update({'font.size': 14})
    a = 0
    config = yaml.safe_load(open("Diagram_Config_T.yml", "r"))

    start = datetime.strptime(config["start_date"], "%d-%m-%Y %H:%M")
    end = datetime.strptime(config["end_date"], "%d-%m-%Y %H:%M")
    time_span = end - start # 1096 days, 23:55:00
    
    for category in config["categories"]:
        readableField = None
        category = config["categories"][category]
        #{'name': 'Rain', 'high_threshold_field': 'precip_type_5min:idx == 1 & precip_5min:mm', 'high_threshold_field_human': 'Rain in mm/5min',
        # 'high_threshold_agg': 'max', 'max_display_value_x': 4, 'min_display_value_x': 0, 'max_display_value_y': 20, 'min_display_value_y': 0, 'thresholds': [0.63, 1.67], 'step_size': 0.1}
        # print("category.keys()") #dict_keys(['name', 'low_threshold_field', 'low_threshold_agg', 'max_display_value_x', 'min_display_value_x', 'max_display_value_y', 'min_display_value_y', 'thresholds', 'step_size'])

        # edit:
        # Statistik des Diagrammes auf ges Datensatz beziehen:
        if "low_threshold_agg" in category.keys() and "high_threshold_agg" in category.keys():
            aggregate = "average"
            if category["low_threshold_field"] == category["high_threshold_field"]:
                field = category["low_threshold_field"]
                if "low_threshold_field_human" in category.keys():
                    readableField = category["low_threshold_field_human"]
                elif "high_threshold_field_human" in category.keys():
                    readableField = category["high_threshold_field_human"]
            else:
                raise ValueError("Different fields for high and low threshold")
        elif "low_threshold_agg" in category.keys():
            aggregate = category["low_threshold_agg"]
            field = category["low_threshold_field"]
            # print(field) #precip_type_5min:idx == 0 & visibility:m
            if "low_threshold_field_human" in category.keys():
                readableField = category["low_threshold_field_human"]
            else:
                readableField = field
        elif "high_threshold_agg" in category.keys():
            aggregate = category["high_threshold_agg"]
            field = category["high_threshold_field"]
            # print(aggregate) # max
            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm

            if "high_threshold_field_human" in category.keys():
                readableField = category["high_threshold_field_human"]
            else:
                readableField = field #precip_type_5min:idx == 0 & visibility:m
        else:
            raise ValueError("No aggregate defined")

        field_min = category["min_display_value_x"]
        field_max = category["max_display_value_x"]
        stepSize = category["step_size"]

        bins = []
        # orig: -> 1. Bin fehlt
        # i = 1
        # edit: -> ok für visibility
        i = 0 # i = 0 damit x Achse ab 0 startet

        while True:
            # newBin = field_min-stepSize + i * stepSize
            newBin = field_min + i * stepSize # ohne NaNs
            if newBin > field_max:
                break
            bins.append(round(newBin, 4))
            i += 1
        print(bins) # [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
        print(stepSize) # 0.1
        print(field_min) # 0
        print(len(bins)) # 41

        numberOfDays = timedelta()
        numberOfDays2 = timedelta()

        result_df = pd.DataFrame([])
        threshold_results = [{"below": timedelta(), "above": timedelta(), "equal": timedelta()} for t in category["thresholds"]]

        for dataFileName in os.listdir(config["file_location"]):
            if not dataFileName.endswith(config["location"]+ "_" + config["area"] + ".parquet"): # HH_vis_rain_Hamburg_all.parquet
                continue
            df = pd.read_parquet(os.path.join(config["file_location"],dataFileName))
            # print(dataFileName) #HH_vis_rain_all_P0_Hamburg_all.parquet
            # print("hhhhhh")
            # print(df.head(100)) # 100 Zeilen von oben
            # print('df Row count is:',len(df.index)) 

            # 3        Hamburg       all  2019-03-03T00:00:00+00:00      min                visibility:m  14036.500000
            # print(df) #5322235       Hamburg       all  2019-11-02T23:55:00+00:00      max  precip_type_5min:idx == 2 & visibility:m            NaN

            # wie soll mit NaNs umgegangen werden? -> löschen oder mit Dummy belegen?

            # print("field")
            # precip_type_5min:idx == 3 & precip_5min:mm
            # precip_type_5min:idx == 0 & visibility:m
            # print(field)
            if "precip_type_5min:idx" in field and "precip_5min:mm" in field:
                print("precip_type_5min")
                df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                # edit für Statistik
                field_min_statistic = 0
                field_max_statistic = 99999
                log = False

                # if "precip_type_5min:idx == 3" in field:
                #     log = True

                # start bin size: -0,2 - 0 -> für NaNs
                # print(bins) # [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0]
                # print(stepSize)
                # print(bins-stepSize)
                # bins = [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
                # [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
                # 0.1
                # print('if df Row count is:',len(df["field_value"].index)) 

                # values = [3, 5, 0, 3, 5, 1, 4, 0, 9]

                # def zero_to_nan(values):
                #     """Replace every 0 with 'nan' and return a copy."""
                #     return [float('nan') if x==0 else x for x in values]
           
            elif "precip_type_5min:idx" in field and "visibility:m" in field:
                print("visibility:m")
                df["field_value"] = df["field_value"].fillna(99999) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                # edit für Statistik
                field_min_statistic = 0
                field_max_statistic = 99999

                # print('if df Row count is:',len(df["field_value"].index)) # falsch
                           
            # df.to_csv("df_before_NaN_visib.csv")
            # df["field_value"].to_csv("df_fieldvalue_before_NaN.csv")
            
            # orig: -> rain, snow
            # df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%

            # df.to_csv("df_after_NaN_visib.csv")

            # print(df.head(100))
            numberOfDays += timedelta(minutes= 5 * len(df["date_time"].unique()))
            #['2018-12-31T00:00:00+00:00' '2018-12-31T00:05:00+00:00'
            #'2018-12-31T00:10:00+00:00' ... '2019-11-02T23:45:00+00:00'
            # '2019-11-02T23:50:00+00:00' '2019-11-02T23:55:00+00:00']
            # print('llll')
            # print(5 * len(df["date_time"].unique()))
            # print("numberOfDays=")
            # print(numberOfDays)
            # print(df["date_time"].unique())

            # print(field_min_statistic) # 0  # 0
            # print(field_max) # 100000 # 2000
            # print("result_df")
            # print(result_df.head(10))
            # print(len(result_df))
            #orig:
            # result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
            #     (df["field_value"] >= field_min) & (df["field_value"] <= field_max)
            #     & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
            result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
                (df["field_value"] >= field_min_statistic) & (df["field_value"] <= field_max_statistic)
                & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
           
            # print(len(result_df))

            # print('result_df Row count is:',len(result_df.index))
            # result_df.to_csv("df_after_NaN_result_rain.csv")
            # print(result_df.head(100))
            #      location_name area_name                  date_time agg_name                                  field_name  field_value
            #         323         Hamburg       all  2018-12-31T00:00:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         659         Hamburg       all  2018-12-31T00:05:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         995         Hamburg       all  2018-12-31T00:10:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            
            # location_name area_name                  date_time agg_name                                field_name  field_value
            # 486227       Hamburg       all  2019-01-05T00:35:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1913.9
            # 486563       Hamburg       all  2019-01-05T00:40:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1772.6
            # 486899       Hamburg       all  2019-01-05T00:45:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1862.0
            # 487235       Hamburg       all  2019-01-05T00:50:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1743.8
    
            # result = pd.cut(df["field_value"],bins=bins, labels=False )

            # for _, row in df.iterrows():
            #     pos = math.floor((row["field_value"] - field_min)/stepSize)
            #     bins[pos] = bins[pos] + 5
        result_df.to_parquet(category["name"] + "_" + config["location"] + ".parquet") #Rain_Hamburg.parquet
        result_df.to_csv(category["name"] + "_" + config["location"] + ".csv")

        # print(5*len(result_df[result_df["field_value"]]))
        # print(result_df[result_df["field_value"]])
        # print(len(result_df["field_value"]))
        # print(len(result_df[result_df["field_value"]]))

        for i,threshold in enumerate(category["thresholds"]):
                
                threshold_results[i]["above"] +=timedelta(minutes=5*len(result_df[result_df["field_value"] > threshold]))
                threshold_results[i]["below"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] < threshold]))
                threshold_results[i]["equal"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] == threshold]))

        # if a== 0:
        #     result_df = pd.read_parquet("Snow_" + config["location"] + ".parquet")
        # else:
        #     result_df = pd.read_parquet("Rain_" + config["location"] + ".parquet")

        # plot NaNs = https://matplotlib.org/stable/gallery/lines_bars_and_markers/masked_demo.html
        with plt.ioff():
            textstr = "Parameter: {}\nAggregate: {}\nLocation: {}\nArea: {}\nStart: {}\nEnd: {}\n".format(
                readableField, aggregate, config["location"], config["area"], config["start_date"], config["end_date"])
            fig, ax = plt.subplots()
            print("plot")
            # print(field_min) # 0
            # orig:
            # ax.set_xlim(left=0-stepSize, right=field_max) # 0,4
            ax.set_xlim(left=0, right=field_max) # 0,4
            # print(0-stepSize) # -0.1
            # ax.set_xlim(left=-0.1, right=field_max) # 0,4
            # print(field_min) # 0
            # print(field_max) # 4
            # print("max_display_value_y" in category.keys())

            if "max_display_value_y" in category.keys() and "min_display_value_y" in category.keys():
                ax.set_ylim(category["min_display_value_y"], category["max_display_value_y"])
                # ax.set_ylim(bottom=category["min_display_value_y"], top=category["max_display_value_y"])
            #ax.stairs(bins, [field_min+ i * stepSize for i in range(0, math.ceil((field_max - field_min)/stepSize)+1)], fill=True)

            # orig -> bottom=True -> y-Werte starten ab 1
            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), bottom=True, align='mid') # left
            # edit
            # print([1/12]*len(result_df.index))
            # print(result_df.index) #[    321,     657,     993,    1329,    1665,    2001,    2337, 2673,    3009,    3345,
            # ...
            # 5415969, 5416305, 5416641, 5416977, 5417313, 5417649, 5417985,
            # 5418321, 5418657, 5418993]
            # print(len(result_df.index)) # 314207

            # print(result_df["field_value"])
            # https://stackoverflow.com/questions/18697417/not-plotting-zero-in-matplotlib-or-change-zero-to-none-python

            
            # def zero_to_nan(values):
            # # def zero_to_nan(result_df["field_value"]):
            #     """Replace every 0 with 'nan' and return a copy."""
            #     return [float('nan') if x==0 else x for x in values]

            # print(zero_to_nan(result_df["field_value"]))
            # print("ffffffff")
            # print(values)
            # print(result_df["field_value"])
            # print(result_df["field_value"])

            result_df["field_value"] = result_df["field_value"].replace(0, -0.01)
            # print(data.replace(0, -0.01))

            #print("after replace")
            # print(result_df["field_value"].head(100))


            # values = result_df["field_value"]
            # def zero_to_nan(values):
            #  # def zero_to_nan(result_df["field_value"]):
            #     """Replace every 0 with 'nan' and return a copy."""
            #     return [float('nan') if x==0 else x for x in values]

            # # print(zero_to_nan(result_df["field_value"]))

            # zero_to_nan(values)
            # print(result_df["field_value"])
            # print("ääääääääääää")
            # print(values)

            # # a = [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1]
            # a = result_df["field_value"]
            # for i, n in enumerate(a):
            #     if n == 0:
            #         a[i] = -0.1
            #         # print(a)


            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # visualisierung NaNs:
            N, bins, patches = ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid')
            # print(patches)

            # for i in range(0,1):
            #     # print(patches[i])
            #     patches[i].set_facecolor('g')
            # # for i in range(3,5):    
            # #     patches[i].set_facecolor('r')
            # for i in range(1, len(patches)):
            #     patches[i].set_facecolor('blue')

            # ax.hist(result_df["field_value"],  align='mid') # left
            # ax.hist(data, bins, weights=[1/12]*len(result_df.index), align='mid') # left

            #plt.xlim(-0.1, 4)
            # plt.show()
            # ax.hist(values, bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # print(bins) # bei i = 1 -> [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(bins) # bei i = 0 -> [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(weights=[1/12]*len(result_df.index))

            # print(len(result_df.index)) # 314200 länge Datensatz
            # print([1/12]*len(result_df.index)) # [0.08333333333333333, 0.08333333333333333, 0.08333333333333333,...]
            if "thresholds" in category.keys():
                for i, threshold in enumerate(category["thresholds"]):
                    ax.axvline(threshold, color="red")
                    plt.text(threshold, ax.get_ylim()[1] *1.05, str(threshold), ha='center', va='center')
                    textstr += "Threshold: {}\n  Below: {}\n  Above: {}\n  Equal: {}\n".format(threshold, threshold_results[i]["below"],threshold_results[i]["above"], threshold_results[i]["equal"] )
            plt.text(0.02, 0.3, textstr, fontsize=14, transform=plt.gcf().transFigure)
            plt.subplots_adjust(left=0.35, bottom=0.15)
            #ax.set_xticks(bins, rotation=90)
            # orig:
            # bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 0]
            # edit:
            bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 1] # The % is called the modulo operator. Of course when the remainder is 0, the number is even.
            plt.xticks(bins2, bins2, rotation=90)
            ax.set_title(" - ".join([config["location"], config["area"]]), pad=30)
            if readableField is not None:
                ax.set_xlabel(readableField)
            else:
                ax.set_xlabel(field)
            ax.set_ylabel("Hours from {} to {}".format(config["start_date"][:10], config["end_date"][:10]))
            # manager = plt.get_current_fig_manager()
            # manager.frame.Maximize(True)
            #plt.plot()
            if log:
                plt.yscale('log')
            #plt.show()
            plt.savefig(category["name"] + "_" + config["location"] + "_.png")
            quit()
            a += 1


def aggregateDiagram_rain_snow():
    plt.rcParams.update({'font.size': 14})
    a = 0
    config = yaml.safe_load(open("Diagram_Config_T.yml", "r"))

    start = datetime.strptime(config["start_date"], "%d-%m-%Y %H:%M")
    end = datetime.strptime(config["end_date"], "%d-%m-%Y %H:%M")
    time_span = end - start # 1096 days, 23:55:00
    
    for category in config["categories"]:
        readableField = None
        category = config["categories"][category]
        #{'name': 'Rain', 'high_threshold_field': 'precip_type_5min:idx == 1 & precip_5min:mm', 'high_threshold_field_human': 'Rain in mm/5min',
        # 'high_threshold_agg': 'max', 'max_display_value_x': 4, 'min_display_value_x': 0, 'max_display_value_y': 20, 'min_display_value_y': 0, 'thresholds': [0.63, 1.67], 'step_size': 0.1}
        # print("category.keys()") #dict_keys(['name', 'low_threshold_field', 'low_threshold_agg', 'max_display_value_x', 'min_display_value_x', 'max_display_value_y', 'min_display_value_y', 'thresholds', 'step_size'])

        # edit:
        # Statistik des Diagrammes auf ges Datensatz beziehen:
        if "low_threshold_agg" in category.keys() and "high_threshold_agg" in category.keys():
            aggregate = "average"
            if category["low_threshold_field"] == category["high_threshold_field"]:
                field = category["low_threshold_field"]
                if "low_threshold_field_human" in category.keys():
                    readableField = category["low_threshold_field_human"]
                elif "high_threshold_field_human" in category.keys():
                    readableField = category["high_threshold_field_human"]
            else:
                raise ValueError("Different fields for high and low threshold")
        elif "low_threshold_agg" in category.keys():
            aggregate = category["low_threshold_agg"]
            field = category["low_threshold_field"]
            # print(field) #precip_type_5min:idx == 0 & visibility:m
            if "low_threshold_field_human" in category.keys():
                readableField = category["low_threshold_field_human"]
            else:
                readableField = field
        elif "high_threshold_agg" in category.keys():
            aggregate = category["high_threshold_agg"]
            field = category["high_threshold_field"]
            # print(aggregate) # max
            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm

            if "high_threshold_field_human" in category.keys():
                readableField = category["high_threshold_field_human"]
            else:
                readableField = field #precip_type_5min:idx == 0 & visibility:m
        else:
            raise ValueError("No aggregate defined")

        field_min = category["min_display_value_x"]
        field_max = category["max_display_value_x"]
        stepSize = category["step_size"]

        bins = []
        # orig: -> 1. Bin fehlt
        # i = 1
        # edit: -> ok für visibility
        i = 0 # i = 0 damit x Achse ab 0 startet

        while True:
            newBin = field_min-stepSize + i * stepSize
            
            if newBin > field_max:
                break
            bins.append(round(newBin, 4))
            i += 1
        print(bins) # [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
        print(stepSize) # 0.1
        print(field_min) # 0
        print(len(bins)) # 41

        numberOfDays = timedelta()
        numberOfDays2 = timedelta()

        result_df = pd.DataFrame([])
        threshold_results = [{"below": timedelta(), "above": timedelta(), "equal": timedelta()} for t in category["thresholds"]]

        for dataFileName in os.listdir(config["file_location"]):
            if not dataFileName.endswith(config["location"]+ "_" + config["area"] + ".parquet"): # HH_vis_rain_Hamburg_all.parquet
                continue
            df = pd.read_parquet(os.path.join(config["file_location"],dataFileName))
            # print(dataFileName) #HH_vis_rain_all_P0_Hamburg_all.parquet
            # print(df.head(100)) # 100 Zeilen von oben
            # print('df Row count is:',len(df.index)) 

            # 3        Hamburg       all  2019-03-03T00:00:00+00:00      min                visibility:m  14036.500000
            # print(df) #5322235       Hamburg       all  2019-11-02T23:55:00+00:00      max  precip_type_5min:idx == 2 & visibility:m            NaN

            # wie soll mit NaNs umgegangen werden? -> löschen oder mit Dummy belegen?

            # print("field")
            # precip_type_5min:idx == 3 & precip_5min:mm
            # precip_type_5min:idx == 0 & visibility:m
            # print(field)
            if "precip_type_5min:idx" in field and "precip_5min:mm" in field:
                print("precip_type_5min")
                df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                # edit für Statistik
                field_min_statistic = 0
                field_max_statistic = 99999
                log = False

                # if "precip_type_5min:idx == 3" in field:
                #     log = True

                # start bin size: -0,2 - 0 -> für NaNs
                # print(bins) # [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0]
                # print(stepSize)
                # print(bins-stepSize)
                # bins = [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
                # [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
                # 0.1
                # print('if df Row count is:',len(df["field_value"].index)) 

                # values = [3, 5, 0, 3, 5, 1, 4, 0, 9]

                # def zero_to_nan(values):
                #     """Replace every 0 with 'nan' and return a copy."""
                #     return [float('nan') if x==0 else x for x in values]
           
            elif "precip_type_5min:idx" in field and "visibility:m" in field:
                print("visibility:m")
                df["field_value"] = df["field_value"].fillna(99999) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                # edit für Statistik
                field_min_statistic = 0
                field_max_statistic = 99999

                # print('if df Row count is:',len(df["field_value"].index)) # falsch
                           
            # df.to_csv("df_before_NaN_visib.csv")
            # df["field_value"].to_csv("df_fieldvalue_before_NaN.csv")
            
            # orig: -> rain, snow
            # df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%

            # df.to_csv("df_after_NaN_visib.csv")

            # print(df.head(100))
            numberOfDays += timedelta(minutes= 5 * len(df["date_time"].unique()))
            #['2018-12-31T00:00:00+00:00' '2018-12-31T00:05:00+00:00'
            #'2018-12-31T00:10:00+00:00' ... '2019-11-02T23:45:00+00:00'
            # '2019-11-02T23:50:00+00:00' '2019-11-02T23:55:00+00:00']
            # print('llll')
            # print(5 * len(df["date_time"].unique()))
            # print("numberOfDays=")
            # print(numberOfDays)
            # print(df["date_time"].unique())

            # print(field_min_statistic) # 0  # 0
            # print(field_max) # 100000 # 2000
            # print("result_df")
            # print(result_df.head(10))
            # print(len(result_df))
            #orig:
            # result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
            #     (df["field_value"] >= field_min) & (df["field_value"] <= field_max)
            #     & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
            result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
                (df["field_value"] >= field_min_statistic) & (df["field_value"] <= field_max_statistic)
                & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
           
            # print(len(result_df))

            # print('result_df Row count is:',len(result_df.index))
            # result_df.to_csv("df_after_NaN_result_rain.csv")
            # print(result_df.head(100))
            #      location_name area_name                  date_time agg_name                                  field_name  field_value
            #         323         Hamburg       all  2018-12-31T00:00:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         659         Hamburg       all  2018-12-31T00:05:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         995         Hamburg       all  2018-12-31T00:10:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            
            # location_name area_name                  date_time agg_name                                field_name  field_value
            # 486227       Hamburg       all  2019-01-05T00:35:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1913.9
            # 486563       Hamburg       all  2019-01-05T00:40:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1772.6
            # 486899       Hamburg       all  2019-01-05T00:45:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1862.0
            # 487235       Hamburg       all  2019-01-05T00:50:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1743.8
    
            # result = pd.cut(df["field_value"],bins=bins, labels=False )

            # for _, row in df.iterrows():
            #     pos = math.floor((row["field_value"] - field_min)/stepSize)
            #     bins[pos] = bins[pos] + 5
        result_df.to_parquet(category["name"] + "_" + config["location"] + ".parquet") #Rain_Hamburg.parquet
        result_df.to_csv(category["name"] + "_" + config["location"] + "neu.csv")

        # print(5*len(result_df[result_df["field_value"]]))
        # print(result_df[result_df["field_value"]])
        # print(len(result_df["field_value"]))
        # print(len(result_df[result_df["field_value"]]))

        for i,threshold in enumerate(category["thresholds"]):
                
                threshold_results[i]["above"] +=timedelta(minutes=5*len(result_df[result_df["field_value"] > threshold]))
                threshold_results[i]["below"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] < threshold]))
                threshold_results[i]["equal"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] == threshold]))

        # if a== 0:
        #     result_df = pd.read_parquet("Snow_" + config["location"] + ".parquet")
        # else:
        #     result_df = pd.read_parquet("Rain_" + config["location"] + ".parquet")

        # plot NaNs = https://matplotlib.org/stable/gallery/lines_bars_and_markers/masked_demo.html
        with plt.ioff():
            textstr = "Parameter: {}\nAggregate: {}\nLocation: {}\nArea: {}\nStart: {}\nEnd: {}\n".format(
                readableField, aggregate, config["location"], config["area"], config["start_date"], config["end_date"])
            fig, ax = plt.subplots()
            print("plot")
            # print(field_min) # 0
            # orig:
            ax.set_xlim(left=0-stepSize, right=field_max) # 0,4
            # print(0-stepSize) # -0.1
            # ax.set_xlim(left=-0.1, right=field_max) # 0,4
            # print(field_min) # 0
            # print(field_max) # 4
            # print("max_display_value_y" in category.keys())

            if "max_display_value_y" in category.keys() and "min_display_value_y" in category.keys():
                ax.set_ylim(category["min_display_value_y"], category["max_display_value_y"])
                # ax.set_ylim(bottom=category["min_display_value_y"], top=category["max_display_value_y"])
            #ax.stairs(bins, [field_min+ i * stepSize for i in range(0, math.ceil((field_max - field_min)/stepSize)+1)], fill=True)

            # orig -> bottom=True -> y-Werte starten ab 1
            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), bottom=True, align='mid') # left
            # edit
            # print([1/12]*len(result_df.index))
            # print(result_df.index) #[    321,     657,     993,    1329,    1665,    2001,    2337, 2673,    3009,    3345,
            # ...
            # 5415969, 5416305, 5416641, 5416977, 5417313, 5417649, 5417985,
            # 5418321, 5418657, 5418993]
            # print(len(result_df.index)) # 314207

            # print(result_df["field_value"])
            # https://stackoverflow.com/questions/18697417/not-plotting-zero-in-matplotlib-or-change-zero-to-none-python

            
            # def zero_to_nan(values):
            # # def zero_to_nan(result_df["field_value"]):
            #     """Replace every 0 with 'nan' and return a copy."""
            #     return [float('nan') if x==0 else x for x in values]

            # print(zero_to_nan(result_df["field_value"]))
            # print("ffffffff")
            # print(values)
            # print(result_df["field_value"])
            # print(result_df["field_value"])

            result_df["field_value"] = result_df["field_value"].replace(0, -0.01)
            # print(data.replace(0, -0.01))

            #print("after replace")
            # print(result_df["field_value"].head(100))


            # values = result_df["field_value"]
            # def zero_to_nan(values):
            #  # def zero_to_nan(result_df["field_value"]):
            #     """Replace every 0 with 'nan' and return a copy."""
            #     return [float('nan') if x==0 else x for x in values]

            # # print(zero_to_nan(result_df["field_value"]))

            # zero_to_nan(values)
            # print(result_df["field_value"])
            # print("ääääääääääää")
            # print(values)

            # # a = [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1]
            # a = result_df["field_value"]
            # for i, n in enumerate(a):
            #     if n == 0:
            #         a[i] = -0.1
            #         # print(a)


            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid') # left

            N, bins, patches = ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid')
            # print(patches)

            for i in range(0,1):
                # print(patches[i])
                patches[i].set_facecolor('g')
            # for i in range(3,5):    
            #     patches[i].set_facecolor('r')
            for i in range(1, len(patches)):
                patches[i].set_facecolor('blue')

            # ax.hist(result_df["field_value"],  align='mid') # left
            # ax.hist(data, bins, weights=[1/12]*len(result_df.index), align='mid') # left

            #plt.xlim(-0.1, 4)
            # plt.show()
            # ax.hist(values, bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # print(bins) # bei i = 1 -> [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(bins) # bei i = 0 -> [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(weights=[1/12]*len(result_df.index))

            # print(len(result_df.index)) # 314200 länge Datensatz
            # print([1/12]*len(result_df.index)) # [0.08333333333333333, 0.08333333333333333, 0.08333333333333333,...]
            if "thresholds" in category.keys():
                for i, threshold in enumerate(category["thresholds"]):
                    ax.axvline(threshold, color="red")
                    plt.text(threshold, ax.get_ylim()[1] *1.05, str(threshold), ha='center', va='center')
                    textstr += "Threshold: {}\n  Below: {}\n  Above: {}\n  Equal: {}\n".format(threshold, threshold_results[i]["below"],threshold_results[i]["above"], threshold_results[i]["equal"] )
            plt.text(0.02, 0.3, textstr, fontsize=14, transform=plt.gcf().transFigure)
            plt.subplots_adjust(left=0.35, bottom=0.15)
            #ax.set_xticks(bins, rotation=90)
            # orig:
            # bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 0]
            # edit:
            bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 1] # The % is called the modulo operator. Of course when the remainder is 0, the number is even.
            plt.xticks(bins2, bins2, rotation=90)
            ax.set_title(" - ".join([config["location"], config["area"]]), pad=30)
            if readableField is not None:
                ax.set_xlabel(readableField)
            else:
                ax.set_xlabel(field)
            ax.set_ylabel("Hours from {} to {}".format(config["start_date"][:10], config["end_date"][:10]))
            # manager = plt.get_current_fig_manager()
            # manager.frame.Maximize(True)
            #plt.plot()
            if log:
                plt.yscale('log')
            #plt.show()
            plt.savefig(category["name"] + "_" + config["location"] + "_.png")
            quit()
            a += 1


def aggregateDiagram_Statistic_all():
    # ignore thresholds
    plt.rcParams.update({'font.size': 14})
    a = 0
    config = yaml.safe_load(open("Diagram_Config_T.yml", "r"))
    for category in config["categories"]:
        readableField = None
        category = config["categories"][category]
        # print("category")
        # print(category)
        #{'name': 'Rain', 'high_threshold_field': 'precip_type_5min:idx == 1 & precip_5min:mm', 'high_threshold_field_human': 'Rain in mm/5min',
        # 'high_threshold_agg': 'max', 'max_display_value_x': 4, 'min_display_value_x': 0, 'max_display_value_y': 20, 'min_display_value_y': 0, 'thresholds': [0.63, 1.67], 'step_size': 0.1}

        # print("category.keys()") #dict_keys(['name', 'low_threshold_field', 'low_threshold_agg', 'max_display_value_x', 'min_display_value_x', 'max_display_value_y', 'min_display_value_y', 'thresholds', 'step_size'])
        # print(category.keys())
        if "low_threshold_agg" in category.keys() and "high_threshold_agg" in category.keys():
            aggregate = "average"
            if category["low_threshold_field"] == category["high_threshold_field"]:
                field = category["low_threshold_field"]
                if "low_threshold_field_human" in category.keys():
                    readableField = category["low_threshold_field_human"]
                elif "high_threshold_field_human" in category.keys():
                    readableField = category["high_threshold_field_human"]
            else:
                raise ValueError("Different fields for high and low threshold")
        elif "low_threshold_agg" in category.keys():
            aggregate = category["low_threshold_agg"]
            field = category["low_threshold_field"]
            # print(field) #precip_type_5min:idx == 0 & visibility:m
            if "low_threshold_field_human" in category.keys():
                readableField = category["low_threshold_field_human"]
            else:
                readableField = field
        elif "high_threshold_agg" in category.keys():
            aggregate = category["high_threshold_agg"]
            field = category["high_threshold_field"]
            # print(aggregate) # max
            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm

            if "high_threshold_field_human" in category.keys():
                readableField = category["high_threshold_field_human"]
            else:
                readableField = field #precip_type_5min:idx == 0 & visibility:m
        else:
            raise ValueError("No aggregate defined")
    

        field_min = category["min_display_value_x"]
        field_max = category["max_display_value_x"]
        stepSize = category["step_size"]

        bins = []
        # orig: -> 1. Bin fehlt
        # i = 1
        # edit:
        i = 0 # i = 0 damit x Achse ab 0 startet

        while True:
            newBin = field_min + i * stepSize
            
            if newBin > field_max:
                break
            bins.append(round(newBin, 4))
            i += 1

        numberOfDays = timedelta()
        numberOfDays2 = timedelta()

        result_df = pd.DataFrame([])
        # threshold_results = [{"below": timedelta(), "above": timedelta(), "equal": timedelta()} for t in category["thresholds"]]

        for dataFileName in os.listdir(config["file_location"]):
            if not dataFileName.endswith(config["location"]+ "_" + config["area"] + ".parquet"): # HH_vis_rain_Hamburg_all.parquet
                continue
            df = pd.read_parquet(os.path.join(config["file_location"],dataFileName))
            # print(dataFileName) #HH_vis_rain_all_P0_Hamburg_all.parquet
            # print("df: ")
            # print(df.head(100)) # 100 Zeilen von oben

            # location_name area_name                  date_time agg_name               field_name    field_value
            # 0        Hamburg       all  2018-12-31T00:00:00+00:00      min                      lat      53.474071
            # 1        Hamburg       all  2018-12-31T00:00:00+00:00      min                      lon       9.872043
            # 2        Hamburg       all  2018-12-31T00:00:00+00:00      min           precip_5min:mm       0.000000
            # 3        Hamburg       all  2018-12-31T00:00:00+00:00      min  effective_cloud_cover:p      79.400000
            # 4        Hamburg       all  2018-12-31T00:00:00+00:00      min          sfc_pressure:Pa  102125.000000

            # print(df) #5322235       Hamburg       all  2019-11-02T23:55:00+00:00      max  precip_type_5min:idx == 2 & visibility:m            NaN

            # wie soll mit NaNs umgegangen werden? -> löschen oder mit Dummy belegen?

            # print("field")
            # precip_type_5min:idx == 3 & precip_5min:mm
            # precip_type_5min:idx == 0 & visibility:m
            # print(field)
            # Dummy Werte setzen:

            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm
            # df.to_csv("df_before_NaN_rain.csv")

            # check = wenn hohe Anzahl an NaNs -> log Darstellung
            # if "precip_type_5min:idx" in field and "precip_5min:mm" in field:
            if "precip_type_5min:idx" in field and "precip_5min:mm" in field:
                print("if = precip_type_5min")
                df["field_value"] = df["field_value"].fillna(0.0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                field_value_NaN = 0
                log = True
            elif "precip_type_5min:idx" in field and "visibility:m" in field:
                print("visibility:m")
                df["field_value"] = df["field_value"].fillna(99999) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                field_value_NaN = 99999
                log = False
            
            # df.to_csv("df_after_NaN_rain.csv")
            # df["field_value"].to_csv("df_fieldvalue_before_NaN.csv")
            
            # orig: -> rain, snow
            # df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%

            # df.to_csv("df_after_NaN_visib.csv")

            # print(df.head(100))
            numberOfDays += timedelta(minutes= 5 * len(df["date_time"].unique()))
            # print(df["date_time"].unique())
            #['2018-12-31T00:00:00+00:00' '2018-12-31T00:05:00+00:00'
            #'2018-12-31T00:10:00+00:00' ... '2019-11-02T23:45:00+00:00'
            # '2019-11-02T23:50:00+00:00' '2019-11-02T23:55:00+00:00']
            # print("field")
            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm
            # print(result_df.head(10))


            # result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
            #     (df["field_value"] >= field_min) & (df["field_value"] <= field_max)
            #     & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])

            result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
                (df["field_value"] >= field_min) & 
                (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])

            
            # print("result_df")
            # result_df.to_csv("df_after_NaN_result_rain.csv")
            # print(type(result_df)) # <class 'pandas.core.frame.DataFrame'>
            # test = result_df.sort_values(by=['field_value'])
            # print(test.head(100))
            # print(result_df.head(10))
            # print(result_df["field_value"].min())
            #      location_name area_name                  date_time agg_name                                  field_name  field_value
            #         323         Hamburg       all  2018-12-31T00:00:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         659         Hamburg       all  2018-12-31T00:05:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         995         Hamburg       all  2018-12-31T00:10:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            
            # location_name area_name                  date_time agg_name                                field_name  field_value
            # 486227       Hamburg       all  2019-01-05T00:35:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1913.9
            # 486563       Hamburg       all  2019-01-05T00:40:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1772.6
            # 486899       Hamburg       all  2019-01-05T00:45:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1862.0
            # 487235       Hamburg       all  2019-01-05T00:50:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1743.8
    
            # result = pd.cut(df["field_value"],bins=bins, labels=False )

            # for _, row in df.iterrows():
            #     pos = math.floor((row["field_value"] - field_min)/stepSize)
            #     bins[pos] = bins[pos] + 5
        result_df.to_parquet(category["name"] + "_" + config["location"] + "_statistic_all.parquet") #Rain_Hamburg.parquet
        result_df.to_csv(category["name"] + "_" + config["location"] + "_statistic_all.csv")
        print(result_df["field_value"].max())
        print(result_df["field_value"].min())
        field_max = result_df["field_value"].max()
        field_min = result_df["field_value"].min()

        # for i,threshold in enumerate(category["thresholds"]):
        #         threshold_results[i]["above"] +=timedelta(minutes=5*len(result_df[result_df["field_value"] > threshold]))
        #         threshold_results[i]["below"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] < threshold]))
        #         threshold_results[i]["equal"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] == threshold]))

        # if a== 0:
        #     result_df = pd.read_parquet("Snow_" + config["location"] + ".parquet")
        # else:
        #     result_df = pd.read_parquet("Rain_" + config["location"] + ".parquet")

        with plt.ioff():
            textstr = "Parameter: {}\nAggregate: {}\nLocation: {}\nArea: {}\nStart: {}\nEnd: {}\nNaN: {}\nMin: {}\nMax: {}\nLog: {}\n".format(
                readableField, aggregate, config["location"], config["area"], config["start_date"], config["end_date"], field_value_NaN, field_min, field_max, log)
            fig, ax = plt.subplots()
            ax.set_xlim(left=field_min, right=field_max) # 0,4

            # if "max_display_value_y" in category.keys() and "min_display_value_y" in category.keys():
            #     ax.set_ylim(bottom=category["min_display_value_y"], top=category["max_display_value_y"])

            #ax.stairs(bins, [field_min+ i * stepSize for i in range(0, math.ceil((field_max - field_min)/stepSize)+1)], fill=True)

            # orig -> bottom=True -> y-Werte starten ab 1
            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), bottom=True, align='mid') # left
            # edit
            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid') # left
            bins = 40
            ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # print(bins) # bei i = 1 -> [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(bins) # bei i = 0 -> [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(weights=[1/12]*len(result_df.index))
            # print(len(result_df.index)) # 314200 länge Datensatz
            # print([1/12]*len(result_df.index)) # [0.08333333333333333, 0.08333333333333333, 0.08333333333333333,...]
            # if "thresholds" in category.keys():
            #     for i, threshold in enumerate(category["thresholds"]):
            #         ax.axvline(threshold, color="red")
            #         plt.text(threshold, ax.get_ylim()[1] *1.05, str(threshold), ha='center', va='center')
            #         textstr += "Threshold: {}\n  Below: {}\n  Above: {}\n  Equal: {}\n".format(threshold, threshold_results[i]["below"],threshold_results[i]["above"], threshold_results[i]["equal"] )
            
            plt.text(0.02, 0.3, textstr, fontsize=14, transform=plt.gcf().transFigure)
            plt.subplots_adjust(left=0.35, bottom=0.15)
            # ax.set_xticks(bins, rotation=90)
            # orig:
            # bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 0]
            # edit:
            # bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 1] # The % is called the modulo operator. Of course when the remainder is 0, the number is even.
            # plt.xticks(bins, bins, rotation=90)
            ax.set_title(" - ".join([config["location"], config["area"]]), pad=30)
            if readableField is not None:
                ax.set_xlabel(readableField)
            else:
                ax.set_xlabel(field)
            ax.set_ylabel("Hours from {} to {}".format(config["start_date"][:10], config["end_date"][:10]))
            # manager = plt.get_current_fig_manager()
            # manager.frame.Maximize(True)
            #plt.plot()

            if log:
                plt.yscale('log')
            # else:
            #     plt.yscale()


            #plt.show()
            plt.savefig(category["name"] + "_" + config["location"] + "_.png")
            quit()
            a += 1

def aggregateDiagram_visibility():
    plt.rcParams.update({'font.size': 14})
    a = 0
    config = yaml.safe_load(open("Diagram_Config_T.yml", "r"))

    start = datetime.strptime(config["start_date"], "%d-%m-%Y %H:%M")
    end = datetime.strptime(config["end_date"], "%d-%m-%Y %H:%M")
    time_span = end - start # 1096 days, 23:55:00
    
    for category in config["categories"]:
        readableField = None
        category = config["categories"][category]
        #{'name': 'Rain', 'high_threshold_field': 'precip_type_5min:idx == 1 & precip_5min:mm', 'high_threshold_field_human': 'Rain in mm/5min',
        # 'high_threshold_agg': 'max', 'max_display_value_x': 4, 'min_display_value_x': 0, 'max_display_value_y': 20, 'min_display_value_y': 0, 'thresholds': [0.63, 1.67], 'step_size': 0.1}
        # print("category.keys()") #dict_keys(['name', 'low_threshold_field', 'low_threshold_agg', 'max_display_value_x', 'min_display_value_x', 'max_display_value_y', 'min_display_value_y', 'thresholds', 'step_size'])

        # edit:
        # Statistik des Diagrammes auf ges Datensatz beziehen:
        if "low_threshold_agg" in category.keys() and "high_threshold_agg" in category.keys():
            aggregate = "average"
            if category["low_threshold_field"] == category["high_threshold_field"]:
                field = category["low_threshold_field"]
                if "low_threshold_field_human" in category.keys():
                    readableField = category["low_threshold_field_human"]
                elif "high_threshold_field_human" in category.keys():
                    readableField = category["high_threshold_field_human"]
            else:
                raise ValueError("Different fields for high and low threshold")
        elif "low_threshold_agg" in category.keys():
            aggregate = category["low_threshold_agg"]
            field = category["low_threshold_field"]
            # print(field) #precip_type_5min:idx == 0 & visibility:m
            if "low_threshold_field_human" in category.keys():
                readableField = category["low_threshold_field_human"]
            else:
                readableField = field
        elif "high_threshold_agg" in category.keys():
            aggregate = category["high_threshold_agg"]
            field = category["high_threshold_field"]
            # print(aggregate) # max
            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm

            if "high_threshold_field_human" in category.keys():
                readableField = category["high_threshold_field_human"]
            else:
                readableField = field #precip_type_5min:idx == 0 & visibility:m
        else:
            raise ValueError("No aggregate defined")

        field_min = category["min_display_value_x"]
        field_max = category["max_display_value_x"]
        stepSize = category["step_size"]

        bins = []
        # orig: -> 1. Bin fehlt
        # i = 1
        # edit: -> ok für visibility
        i = 0 # i = 0 damit x Achse ab 0 startet

        while True:
            newBin = field_min + i * stepSize
            
            if newBin > field_max:
                break
            bins.append(round(newBin, 4))
            i += 1
        # print(bins) # [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0]
        # print(stepSize)

        numberOfDays = timedelta()
        numberOfDays2 = timedelta()

        result_df = pd.DataFrame([])
        threshold_results = [{"below": timedelta(), "above": timedelta(), "equal": timedelta()} for t in category["thresholds"]]

        for dataFileName in os.listdir(config["file_location"]):
            if not dataFileName.endswith(config["location"]+ "_" + config["area"] + ".parquet"): # HH_vis_rain_Hamburg_all.parquet
                continue
            df = pd.read_parquet(os.path.join(config["file_location"],dataFileName))
            # print(dataFileName) #HH_vis_rain_all_P0_Hamburg_all.parquet
            # print(df.head(100)) # 100 Zeilen von oben
            # print('df Row count is:',len(df.index)) 

            # 3        Hamburg       all  2019-03-03T00:00:00+00:00      min                visibility:m  14036.500000
            # print(df) #5322235       Hamburg       all  2019-11-02T23:55:00+00:00      max  precip_type_5min:idx == 2 & visibility:m            NaN

            # wie soll mit NaNs umgegangen werden? -> löschen oder mit Dummy belegen?

            # print("field")
            # precip_type_5min:idx == 3 & precip_5min:mm
            # precip_type_5min:idx == 0 & visibility:m
            # print(field)
            if "precip_type_5min:idx" in field and "precip_5min:mm" in field:
                print("precip_type_5min")
                df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                # edit für Statistik
                field_min_statistic = 0
                field_max_statistic = 99999
                # start bin size: -0,2 - 0 -> für NaNs
                # print(bins) # [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0]
                # print(stepSize)
                # print(bins-stepSize)
                # bins = [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
                # [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
                # 0.1
                # print('if df Row count is:',len(df["field_value"].index)) 

                # values = [3, 5, 0, 3, 5, 1, 4, 0, 9]

                # def zero_to_nan(values):
                #     """Replace every 0 with 'nan' and return a copy."""
                #     return [float('nan') if x==0 else x for x in values]

            # elif "precip_type_5min:idx" in field and "visibility:m" in field:
            elif "visibility:m" in field:
                print("visibility:m")
                df["field_value"] = df["field_value"].fillna(99999) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
                # edit für Statistik
                field_min_statistic = 0
                field_max_statistic = 99999

                # print('if df Row count is:',len(df["field_value"].index)) # falsch
                           
            # df.to_csv("df_before_NaN_visib.csv")
            # df["field_value"].to_csv("df_fieldvalue_before_NaN.csv")
            
            # orig: -> rain, snow
            # df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%

            # df.to_csv("df_after_NaN_visib.csv")

            # print(df.head(100))
            numberOfDays += timedelta(minutes= 5 * len(df["date_time"].unique()))
            #['2018-12-31T00:00:00+00:00' '2018-12-31T00:05:00+00:00'
            #'2018-12-31T00:10:00+00:00' ... '2019-11-02T23:45:00+00:00'
            # '2019-11-02T23:50:00+00:00' '2019-11-02T23:55:00+00:00']
            # print('llll')
            # print(5 * len(df["date_time"].unique()))
            # print("numberOfDays=")
            # print(numberOfDays)
            # print(df["date_time"].unique())

            # print(field_min_statistic) # 0  # 0
            # print(field_max) # 100000 # 2000
            # print("result_df")
            # print(result_df.head(10))
            # print(len(result_df))
            #orig:
            # result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
            #     (df["field_value"] >= field_min) & (df["field_value"] <= field_max)
            #     & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
            result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
                (df["field_value"] >= field_min_statistic) & (df["field_value"] <= field_max_statistic)
                & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
           
            # print(len(result_df))

            # print('result_df Row count is:',len(result_df.index))
            # result_df.to_csv("df_after_NaN_result_rain.csv")
            # print(result_df.head(100))
            #      location_name area_name                  date_time agg_name                                  field_name  field_value
            #         323         Hamburg       all  2018-12-31T00:00:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         659         Hamburg       all  2018-12-31T00:05:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         995         Hamburg       all  2018-12-31T00:10:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            
            # location_name area_name                  date_time agg_name                                field_name  field_value
            # 486227       Hamburg       all  2019-01-05T00:35:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1913.9
            # 486563       Hamburg       all  2019-01-05T00:40:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1772.6
            # 486899       Hamburg       all  2019-01-05T00:45:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1862.0
            # 487235       Hamburg       all  2019-01-05T00:50:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1743.8
    
            # result = pd.cut(df["field_value"],bins=bins, labels=False )

            # for _, row in df.iterrows():
            #     pos = math.floor((row["field_value"] - field_min)/stepSize)
            #     bins[pos] = bins[pos] + 5
        result_df.to_parquet(category["name"] + "_" + config["location"] + ".parquet") #Rain_Hamburg.parquet
        result_df.to_csv(category["name"] + "_" + config["location"] + ".csv")

        # print(5*len(result_df[result_df["field_value"]]))
        # print(result_df[result_df["field_value"]])
        # print(len(result_df["field_value"]))
        # print(len(result_df[result_df["field_value"]]))

        for i,threshold in enumerate(category["thresholds"]):
                
                threshold_results[i]["above"] +=timedelta(minutes=5*len(result_df[result_df["field_value"] > threshold]))
                threshold_results[i]["below"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] < threshold]))
                threshold_results[i]["equal"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] == threshold]))

        # if a== 0:
        #     result_df = pd.read_parquet("Snow_" + config["location"] + ".parquet")
        # else:
        #     result_df = pd.read_parquet("Rain_" + config["location"] + ".parquet")

        # plot NaNs = https://matplotlib.org/stable/gallery/lines_bars_and_markers/masked_demo.html
        with plt.ioff():
            textstr = "Parameter: {}\nAggregate: {}\nLocation: {}\nArea: {}\nStart: {}\nEnd: {}\n".format(
                readableField, aggregate, config["location"], config["area"], config["start_date"], config["end_date"])
            fig, ax = plt.subplots()
            # orig:
            ax.set_xlim(left=field_min, right=field_max) # 0,4
            # ax.set_xlim(left=-0.1, right=field_max) # 0,4
            # print(field_min) # 0
            # print(field_max) # 4
            # print("max_display_value_y" in category.keys())

            if "max_display_value_y" in category.keys() and "min_display_value_y" in category.keys():
                ax.set_ylim(bottom=category["min_display_value_y"], top=category["max_display_value_y"])
            #ax.stairs(bins, [field_min+ i * stepSize for i in range(0, math.ceil((field_max - field_min)/stepSize)+1)], fill=True)

            # orig -> bottom=True -> y-Werte starten ab 1
            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), bottom=True, align='mid') # left
            # edit
            # print([1/12]*len(result_df.index))
            # print(result_df.index) #[    321,     657,     993,    1329,    1665,    2001,    2337, 2673,    3009,    3345,
            # ...
            # 5415969, 5416305, 5416641, 5416977, 5417313, 5417649, 5417985,
            # 5418321, 5418657, 5418993]
            # print(len(result_df.index)) # 314207

            # print(result_df["field_value"])
            # https://stackoverflow.com/questions/18697417/not-plotting-zero-in-matplotlib-or-change-zero-to-none-python

            # # values = [3, 5, 0, 3, 5, 1, 4, 0, 9]
            # values = result_df["field_value"]

            # def zero_to_nan(values):
            # # def zero_to_nan(result_df["field_value"]):
            #     """Replace every 0 with 'nan' and return a copy."""
            #     return [float('nan') if x==0 else x for x in values]

            # print(zero_to_nan(result_df["field_value"]))
            # print("ffffffff")
            # print(values)


            ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # ax.hist(values, bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # print(bins) # bei i = 1 -> [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(bins) # bei i = 0 -> [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(weights=[1/12]*len(result_df.index))

            # print(len(result_df.index)) # 314200 länge Datensatz
            # print([1/12]*len(result_df.index)) # [0.08333333333333333, 0.08333333333333333, 0.08333333333333333,...]
            if "thresholds" in category.keys():
                for i, threshold in enumerate(category["thresholds"]):
                    ax.axvline(threshold, color="red")
                    plt.text(threshold, ax.get_ylim()[1] *1.05, str(threshold), ha='center', va='center')
                    textstr += "Threshold: {}\n  Below: {}\n  Above: {}\n  Equal: {}\n".format(threshold, threshold_results[i]["below"],threshold_results[i]["above"], threshold_results[i]["equal"] )
            plt.text(0.02, 0.3, textstr, fontsize=14, transform=plt.gcf().transFigure)
            plt.subplots_adjust(left=0.35, bottom=0.15)
            #ax.set_xticks(bins, rotation=90)
            # orig:
            # bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 0]
            # edit:
            bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 1] # The % is called the modulo operator. Of course when the remainder is 0, the number is even.
            plt.xticks(bins2, bins2, rotation=90)
            ax.set_title(" - ".join([config["location"], config["area"]]), pad=50)
            if readableField is not None:
                ax.set_xlabel(readableField)
            else:
                ax.set_xlabel(field)
            ax.set_ylabel("Hours from {} to {}".format(config["start_date"][:10], config["end_date"][:10]))
            # manager = plt.get_current_fig_manager()
            # manager.frame.Maximize(True)
            #plt.plot()
            #plt.show()
            plt.savefig(category["name"] + "_" + config["location"] + "_.png")
            quit()
            a += 1


def aggregateDiagram_origJohannes():
    plt.rcParams.update({'font.size': 14})
    a = 0
    config = yaml.safe_load(open("Diagram_Config_T.yml", "r"))
    for category in config["categories"]:
        readableField = None
        category = config["categories"][category]
        # print("category")
        # print(category)
        #{'name': 'Rain', 'high_threshold_field': 'precip_type_5min:idx == 1 & precip_5min:mm', 'high_threshold_field_human': 'Rain in mm/5min',
        # 'high_threshold_agg': 'max', 'max_display_value_x': 4, 'min_display_value_x': 0, 'max_display_value_y': 20, 'min_display_value_y': 0, 'thresholds': [0.63, 1.67], 'step_size': 0.1}

        # print("category.keys()") #dict_keys(['name', 'low_threshold_field', 'low_threshold_agg', 'max_display_value_x', 'min_display_value_x', 'max_display_value_y', 'min_display_value_y', 'thresholds', 'step_size'])
        # print(category.keys())
        if "low_threshold_agg" in category.keys() and "high_threshold_agg" in category.keys():
            aggregate = "average"
            if category["low_threshold_field"] == category["high_threshold_field"]:
                field = category["low_threshold_field"]
                if "low_threshold_field_human" in category.keys():
                    readableField = category["low_threshold_field_human"]
                elif "high_threshold_field_human" in category.keys():
                    readableField = category["high_threshold_field_human"]
            else:
                raise ValueError("Different fields for high and low threshold")
        elif "low_threshold_agg" in category.keys():
            # print("low_threshold_agg")
            aggregate = category["low_threshold_agg"]
            field = category["low_threshold_field"]
            # print("field")
            # print(field) #precip_type_5min:idx == 0 & visibility:m
            if "low_threshold_field_human" in category.keys():
                readableField = category["low_threshold_field_human"]
            else:
                readableField = field
        elif "high_threshold_agg" in category.keys():
            aggregate = category["high_threshold_agg"]
            field = category["high_threshold_field"]
            # print(aggregate) # max
            # print(field) # precip_type_5min:idx == 1 & precip_5min:mm

            if "high_threshold_field_human" in category.keys():
                readableField = category["high_threshold_field_human"]
            else:
                readableField = field #precip_type_5min:idx == 0 & visibility:m
        else:
            raise ValueError("No aggregate defined")

        field_min = category["min_display_value_x"]
        field_max = category["max_display_value_x"]
        stepSize = category["step_size"]

        bins = []
        # orig: -> 1. Bin fehlt
        # i = 1
        # edit:
        i = 0 # i = 0 damit x Achse ab 0 startet

        while True:
            newBin = field_min + i * stepSize
            
            if newBin > field_max:
                break
            bins.append(round(newBin, 4))
            i += 1

        numberOfDays = timedelta()
        numberOfDays2 = timedelta()

        result_df = pd.DataFrame([])
        threshold_results = [{"below": timedelta(), "above": timedelta(), "equal": timedelta()} for t in category["thresholds"]]

        for dataFileName in os.listdir(config["file_location"]):
            if not dataFileName.endswith(config["location"]+ "_" + config["area"] + ".parquet"): # HH_vis_rain_Hamburg_all.parquet
                continue
            df = pd.read_parquet(os.path.join(config["file_location"],dataFileName))
            # print(dataFileName) #HH_vis_rain_all_P0_Hamburg_all.parquet
            print(df.head(100)) # 100 Zeilen von oben
            # 3        Hamburg       all  2019-03-03T00:00:00+00:00      min                visibility:m  14036.500000
            # print(df) #5322235       Hamburg       all  2019-11-02T23:55:00+00:00      max  precip_type_5min:idx == 2 & visibility:m            NaN

            # wie soll mit NaNs umgegangen werden? -> löschen oder mit Dummy belegen?

            # print("field")
            # precip_type_5min:idx == 3 & precip_5min:mm
            # precip_type_5min:idx == 0 & visibility:m
            print(field)
            if "precip_type_5min:idx" in field and "precip_5min:mm" in field:
                print("precip_type_5min")
                df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%
            elif "precip_type_5min:idx" in field and "visibility:m" in field:
                print("visibility:m")
                df["field_value"] = df["field_value"].fillna(99999) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%

            # df.to_csv("df_before_NaN_visib.csv")
            # df["field_value"].to_csv("df_fieldvalue_before_NaN.csv")
            
            # orig: -> rain, snow
            # df["field_value"] = df["field_value"].fillna(0) # rain NaNs -> ca. 70%, snow => ca. 98%, visibility = 3%

            # df.to_csv("df_after_NaN_visib.csv")

            # print(df.head(100))
            numberOfDays += timedelta(minutes= 5 * len(df["date_time"].unique()))
            # print(df["date_time"].unique())
            #['2018-12-31T00:00:00+00:00' '2018-12-31T00:05:00+00:00'
            #'2018-12-31T00:10:00+00:00' ... '2019-11-02T23:45:00+00:00'
            # '2019-11-02T23:50:00+00:00' '2019-11-02T23:55:00+00:00']

            result_df = pd.concat([result_df, df[(df["field_name"]==field) & (df["agg_name"] == aggregate) &
                (df["field_value"] >= field_min) & (df["field_value"] <= field_max)
                & (df["location_name"] == config["location"]) & (df["area_name"] ==config["area"])]])
            
            #   print("result_df")
            # result_df.to_csv("df_after_NaN_result_rain.csv")
            # print(result_df.head(100))
            #      location_name area_name                  date_time agg_name                                  field_name  field_value
            #         323         Hamburg       all  2018-12-31T00:00:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         659         Hamburg       all  2018-12-31T00:05:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            #         995         Hamburg       all  2018-12-31T00:10:00+00:00      max  precip_type_5min:idx == 3 & precip_5min:mm          0.0
            
            # location_name area_name                  date_time agg_name                                field_name  field_value
            # 486227       Hamburg       all  2019-01-05T00:35:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1913.9
            # 486563       Hamburg       all  2019-01-05T00:40:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1772.6
            # 486899       Hamburg       all  2019-01-05T00:45:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1862.0
            # 487235       Hamburg       all  2019-01-05T00:50:00+00:00      min  precip_type_5min:idx == 0 & visibility:m       1743.8
    
            # result = pd.cut(df["field_value"],bins=bins, labels=False )

            # for _, row in df.iterrows():
            #     pos = math.floor((row["field_value"] - field_min)/stepSize)
            #     bins[pos] = bins[pos] + 5
        result_df.to_parquet(category["name"] + "_" + config["location"] + ".parquet") #Rain_Hamburg.parquet
        result_df.to_csv(category["name"] + "_" + config["location"] + "neu.csv")

        for i,threshold in enumerate(category["thresholds"]):
                threshold_results[i]["above"] +=timedelta(minutes=5*len(result_df[result_df["field_value"] > threshold]))
                threshold_results[i]["below"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] < threshold]))
                threshold_results[i]["equal"] += timedelta(minutes=5 * len(result_df[result_df["field_value"] == threshold]))

        # if a== 0:
        #     result_df = pd.read_parquet("Snow_" + config["location"] + ".parquet")
        # else:
        #     result_df = pd.read_parquet("Rain_" + config["location"] + ".parquet")

        with plt.ioff():
            textstr = "Parameter: {}\nAggregate: {}\nLocation: {}\nArea: {}\nStart: {}\nEnd: {}\n".format(
                readableField, aggregate, config["location"], config["area"], config["start_date"], config["end_date"])
            fig, ax = plt.subplots()
            ax.set_xlim(left=field_min, right=field_max) # 0,4

            if "max_display_value_y" in category.keys() and "min_display_value_y" in category.keys():
                ax.set_ylim(bottom=category["min_display_value_y"], top=category["max_display_value_y"])
            #ax.stairs(bins, [field_min+ i * stepSize for i in range(0, math.ceil((field_max - field_min)/stepSize)+1)], fill=True)

            # orig -> bottom=True -> y-Werte starten ab 1
            # ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), bottom=True, align='mid') # left
            # edit
            ax.hist(result_df["field_value"], bins, weights=[1/12]*len(result_df.index), align='mid') # left
            # print(bins) # bei i = 1 -> [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(bins) # bei i = 0 -> [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0]
            # print(weights=[1/12]*len(result_df.index))
            # print(len(result_df.index)) # 314200 länge Datensatz
            # print([1/12]*len(result_df.index)) # [0.08333333333333333, 0.08333333333333333, 0.08333333333333333,...]
            if "thresholds" in category.keys():
                for i, threshold in enumerate(category["thresholds"]):
                    ax.axvline(threshold, color="red")
                    plt.text(threshold, ax.get_ylim()[1] *1.05, str(threshold), ha='center', va='center')
                    textstr += "Threshold: {}\n  Below: {}\n  Above: {}\n  Equal: {}\n".format(threshold, threshold_results[i]["below"],threshold_results[i]["above"], threshold_results[i]["equal"] )
            plt.text(0.02, 0.3, textstr, fontsize=14, transform=plt.gcf().transFigure)
            plt.subplots_adjust(left=0.35, bottom=0.15)
            #ax.set_xticks(bins, rotation=90)
            # orig:
            # bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 0]
            # edit:
            bins2 = [bin for i,bin in enumerate(bins) if i % 2 != 1] # The % is called the modulo operator. Of course when the remainder is 0, the number is even.
            plt.xticks(bins2, bins2, rotation=90)
            ax.set_title(" - ".join([config["location"], config["area"]]), pad=30)
            if readableField is not None:
                ax.set_xlabel(readableField)
            else:
                ax.set_xlabel(field)
            ax.set_ylabel("Hours from {} to {}".format(config["start_date"][:10], config["end_date"][:10]))
            # manager = plt.get_current_fig_manager()
            # manager.frame.Maximize(True)
            #plt.plot()
            #plt.show()
            plt.savefig(category["name"] + "_" + config["location"] + "_.png")
            quit()
            a += 1

# Helferfunktion von aggregateTime()
def collect(file):
    df = pd.DataFrame(columns=["Location", "Area", "Event", "Start", "End", "Below/above"])
    thresholdLinesAfterEvent = 0
    currEntry = {}
    for line in file:
        result = re.match(eventStartPattern, line)
        if result is not None:
            currEntry["Location"] = result.group(1)
            currEntry["Area"] = result.group(2)
            currEntry["Event"] = result.group(3)
            currEntry["Start"] = result.group(4)
            thresholdLinesAfterEvent = 2
        elif thresholdLinesAfterEvent > 0:
            result = re.match(thresholdPattern, line)
            if result is None:
                continue
            if thresholdLinesAfterEvent == 2:
                try:
                    if float(result.group(1)) <= float(result.group(2)):
                        currEntry["Below/above"] = "below"
                except ValueError:
                    pass
            elif thresholdLinesAfterEvent == 1:
                try:
                    if float(result.group(1)) >= float(result.group(2)):
                        currEntry["Below/above"] = "above"
                except ValueError:
                    pass
            thresholdLinesAfterEvent = thresholdLinesAfterEvent - 1
        elif currEntry != {}:
            result = re.match(eventEndPattern, line)
            if result is not None:
                currEntry["End"] = result.group(4)
                currEntry = {k: [v] for k,v in currEntry.items()}
                df = pd.concat([df, pd.DataFrame.from_dict(currEntry)], ignore_index=True)
                currEntry = {}
    return df

def getDate(date_str):
    # Correct the offset if there is only one digit for hour
    # we need two cases as the offset can be stated with or without colons
    if re.search(r"(\+|-)\d:\d\d$", date_str) is not None:
        date_str = date_str[:-4] + "0" + date_str[-4:]
    elif re.search(r"(\+|-)\d\d\d$", date_str) is not None:
        date_str = date_str[:-3] + "0" + date_str[-3:]
    if re.search(r"(\+|-)\d\d:?\d\d$", date_str) is None:
        raise ValueError("Unknown way of expressing offset in " + date_str )
    try:
        date = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S %Z%z')
    except ValueError:
        date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S%z")
    return date

def aggregateTime():
    plt.rcParams.update({'font.size': 14})

    config = yaml.safe_load(open("Diagram_Config_T.yml", "r"))
    location = config["location"]
    area = config["area"]
    # print(config["events"])
    # {'event_rain': {'name': 'Rain Event', 'mode': 'month', 'high_threshold_field': 'precip_type_5min:idx == 1 & precip_5min:mm', 'high_threshold_agg': 'max', 'high_threshold_value': 1.67, 'downtime_before': '01:00:00', 'downtime_after': '01:00:00', 'downTime': False, 'direction': 'above'}}
    
    # {'event_visibility_noPrec': {'name': 'Visibility', 'mode': 'month',
    # 'low_threshold_field': 'precip_type_5min:idx == 0 & visibility:m',
    # 'low_threshold_agg': 'min', 'low_threshold_value': 244, 'downtime_before': '01:00:00',
    # 'downtime_after': '01:00:00', 'downTime': True, 'direction': 'below'}}

    for event in config["events"]:
        monthDistribution = {x+1 : 0 for x in range(0,12)}
        hourDistribution = {x: 0 for x in range(0,24)}
        # print(monthDistribution) # {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}
        # {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}
        # print(config["events"][event]["downTime"]) # True or False
        if config["events"][event]["downTime"]:
            downtimeIncluded = "with_downtime"
        else:
            downtimeIncluded = "no_downtime"
        duration = timedelta()
        # print(duration) # 0:00:00
        # print(os.listdir(config["file_location"]))
        # ['HH_vis_rain_all_P0.txt', 'HH_vis_rain_all_P0_events.parquet', 'HH_vis_rain_all_P0_Hamburg_all.parquet',
        # 'HH_vis_rain_all_P1.txt', 'HH_vis_rain_all_P10.txt', 'HH_vis_rain_all_P10_events.parquet',
        # 'HH_vis_rain_all_P10_Hamburg_all.parquet', 'HH_vis_rain_all_P11.txt', 'HH_vis_rain_all_P11_events.parquet',
        # 'HH_vis_rain_all_P11_Hamburg_all.parquet', 'HH_vis_rain_all_P12.txt', 'HH_vis_rain_all_P12_events.parquet',
        # 'HH_vis_rain_all_P12_Hamburg_all.parquet', 'HH_vis_rain_all_P13.txt', 'HH_vis_rain_all_P13_events.parquet',
        # 'HH_vis_rain_all_P13_Hamburg_all.parquet', 'HH_vis_rain_all_P14.txt', 'HH_vis_rain_all_P14_events.parquet',
        #  'HH_vis_rain_all_P14_Hamburg_all.parquet', 'HH_vis_rain_all_P15.txt', 'HH_vis_rain_all_P15_events.parquet',
        # 'HH_vis_rain_all_P15_Hamburg_all.parquet', 'HH_vis_rain_all_P16.txt', 'HH_vis_rain_all_P16_events.parquet',
        #  'HH_vis_rain_all_P16_Hamburg_all.parquet', 'HH_vis_rain_all_P17.txt', 'HH_vis_rain_all_P17_events.parquet',
        #  'HH_vis_rain_all_P17_Hamburg_all.parquet', 'HH_vis_rain_all_P18.txt', 'HH_vis_rain_all_P18_events.parquet',
        #  'HH_vis_rain_all_P18_Hamburg_all.parquet', 'HH_vis_rain_all_P19.txt', 'HH_vis_rain_all_P19_events.parquet',
        #  'HH_vis_rain_all_P19_Hamburg_all.parquet', 'HH_vis_rain_all_P1_events.parquet',
        # 'HH_vis_rain_all_P1_Hamburg_all.parquet', 'HH_vis_rain_all_P2.txt', 'HH_vis_rain_all_P2_events.parquet',
        # 'HH_vis_rain_all_P2_Hamburg_all.parquet', 'HH_vis_rain_all_P3.txt', 'HH_vis_rain_all_P3_events.parquet',
        # 'HH_vis_rain_all_P3_Hamburg_all.parquet', 'HH_vis_rain_all_P4.txt', 'HH_vis_rain_all_P4_events.parquet',
        # 'HH_vis_rain_all_P4_Hamburg_all.parquet', 'HH_vis_rain_all_P5.txt', 'HH_vis_rain_all_P5_events.parquet',
        # 'HH_vis_rain_all_P5_Hamburg_all.parquet', 'HH_vis_rain_all_P6.txt', 'HH_vis_rain_all_P6_events.parquet',
        # 'HH_vis_rain_all_P6_Hamburg_all.parquet', 'HH_vis_rain_all_P7.txt', 'HH_vis_rain_all_P7_events.parquet',
        # 'HH_vis_rain_all_P7_Hamburg_all.parquet', 'HH_vis_rain_all_P8.txt', 'HH_vis_rain_all_P8_events.parquet',
        # 'HH_vis_rain_all_P8_Hamburg_all.parquet', 'HH_vis_rain_all_P9.txt', 'HH_vis_rain_all_P9_events.parquet',
        #  'HH_vis_rain_all_P9_Hamburg_all.parquet']
        for dataFileName in os.listdir(config["file_location"]):
            if not dataFileName.endswith("events.parquet"):
                continue

            print(dataFileName.rsplit("_", 1)[0] + ".txt") #HH_vis_rain_all_P0.txt
            with open(os.path.join(config["file_location"], dataFileName.rsplit("_", 1)[0] + ".txt")) as f:
                logEvents = collect(f)
                # print("logEvents1")
                # print(logEvents)
                    # Location Area                     Event                Start                  End Below/above
                    # 0    Hamburg  all         event_temperature  2018-12-31T00:00:00  2018-12-31T00:15:00       below
                    # 1    Hamburg  all         event_temperature  2019-01-01T16:05:00  2019-01-15T09:45:00       below
                    # 2    Hamburg  all         event_temperature  2019-01-19T03:40:00  2019-01-04T14:50:00       below
                    # 3    Hamburg  all         event_temperature  2019-01-22T15:00:00  2019-01-22T15:00:00       below
                    # 4    Hamburg  all         event_temperature  2019-01-05T16:05:00  2019-01-06T00:00:00       below

                logEvents = logEvents[(logEvents["Area"] == area) & (logEvents["Location"] == location)
                                    & (logEvents["Event"] == event)]
                
                print("logEvents2")
                print(event) # event_rain
                print(logEvents["Event"])
                print(logEvents)
                #   Location Area         Event                Start                  End Below/above
                #     19  Hamburg  all  event_rain_a  2019-01-13T02:45:00  2019-01-13T02:50:00       above
                #     20  Hamburg  all  event_rain_a  2019-01-13T06:05:00  2019-01-13T06:10:00       above
                #     21  Hamburg  all  event_rain_a  2019-01-13T06:35:00  2019-01-13T06:50:00       above
                #     22  Hamburg  all  event_rain_a  2019-01-13T14:20:00  2019-01-13T15:20:00       above
                #     23  Hamburg  all  event_rain_a  2019-01-13T16:40:00  2019-01-13T16:45:00       above

            df = pd.read_parquet(os.path.join(config["file_location"],dataFileName))
            # print(df)
            #               type location_name area_name                event_name                 start_datetime                   end_datetime
            # 0    with_downtime       Hamburg       all         event_temperature  2018-12-31T00:00:00 UTC+01:00  2018-12-31T02:15:00 UTC+01:00
            # 1    with_downtime       Hamburg       all         event_temperature  2019-01-01T16:05:00 UTC+01:00  2019-01-15T11:45:00 UTC+01:00
            # 2    with_downtime       Hamburg       all         event_temperature  2019-01-05T16:05:00 UTC+01:00  2019-01-06T02:00:00 UTC+01:00
            # 3    with_downtime       Hamburg       all         event_temperature  2019-01-06T23:15:00 UTC+01:00  2019-01-07T03:15:00 UTC+01:00
            # 4    with_downtime       Hamburg       all         event_temperature  2019-01-08T15:05:00 UTC+01:00  2019-01-10T08:00:00 UTC+01:00
            
            # print(len(df.index)) # 223, 251,367


            df = df[(df["location_name"] == location) & ( df["area_name"] == area)
                & (df["type"] == downtimeIncluded) & (df["event_name"] == event)]
            
            if len(df.index) == 0:
                print('NO ELIGIBLE ROWS FOR FILE')
                continue
                
                #         type location_name area_name    event_name                 start_datetime                   end_datetime
                # 87   no_downtime       Hamburg       all  event_rain_a  2019-01-07T22:55:00 UTC+01:00  2019-01-07T23:00:00 UTC+01:00
                # 88   no_downtime       Hamburg       all  event_rain_a  2019-01-08T03:30:00 UTC+01:00  2019-01-08T04:10:00 UTC+01:00
                # 89   no_downtime       Hamburg       all  event_rain_a  2019-01-08T04:15:00 UTC+01:00  2019-01-08T04:20:00 UTC+01:00
                # 90   no_downtime       Hamburg       all  event_rain_a  2019-01-08T04:45:00 UTC+01:00  2019-01-08T05:20:00 UTC+01:00
                # 91   no_downtime       Hamburg       all  event_rain_a  2019-01-08T05:45:00 UTC+01:00  2019-01-08T06:15:00 UTC+01:00


            # add 5 minutes to current time starting from event start until end
            # localise and get month/hour
            # print("df ->")
            # print(df) # Empty DataFrame
            # print("before for")
            for _, row in df.iterrows():
                # print("after for")
                # start = datetime.strptime(row["start_datetime"], "%Y-%m-%dT%H:%M:%S %Z%z")
                # end = datetime.strptime(row["end_datetime"], "%Y-%m-%dT%H:%M:%S %Z%z")
                start = getDate(row["start_datetime"])
                end = getDate(row["end_datetime"])

                # print(start)
                # HH_vis_rain_all_P0.txt
                # 2019-01-07 22:55:00+01:00
                # 2019-01-08 03:30:00+01:00
                # 2019-01-08 04:15:00+01:00
                # 2019-01-08 04:45:00+01:00
                # 2019-01-08 05:45:00+01:00
                # 2019-01-08 06:40:00+01:00
                # print(end)
                # 2019-01-07 23:00:00+01:00
                # 2019-01-08 04:10:00+01:00
                # 2019-01-08 04:20:00+01:00

                # XXX POTENTIAL BIG BUG: this might count each event once per (5 min?) time interval within it!
                direction = logEvents[(logEvents["Start"] >= (start-start.utcoffset()).strftime("%Y-%m-%dT%H:%M:%S")) &
                          (logEvents["End"] <= (end - end.utcoffset()).strftime("%Y-%m-%dT%H:%M:%S"))]["Below/above"].unique()
                # print(logEvents["Start"])
                # 19    2019-01-13T02:45:00
                # 20    2019-01-13T06:05:00
                # 21    2019-01-13T06:35:00

                # print((start-start.utcoffset()).strftime("%Y-%m-%dT%H:%M:%S"))
                # 2019-01-07T21:55:00
                # 2019-01-08T02:30:00
                # 2019-01-08T03:15:00
                # 2019-01-08T03:45:00

                # print(direction) 
                # ['above']
                # ['above']

                if len(direction) != 1:
                    # print(row)
                    # print("multiple directions")
                    continue
                else:
                    direction = direction[0]
                if direction != config["events"][event]["direction"]:
                    # print("wrong direction")
                    continue

                duration += end -start
                # print(duration)
                # 0:05:00
                # 0:45:00
                # 0:50:00
                # 1:25:00
                # 1:55:00
                # 2:50:00

                minutes = 0
                curr = start
                while curr < end:
                    monthDistribution[curr.month] += 5
                    # print(monthDistribution[curr.month])
                    # 5
                    # 10
                    # 15
                    # 20
                    # 25
                    # 30
                    # 35
                    # 40
                    hourDistribution[curr.hour] +=5
                    # print(hourDistribution[curr.hour])
                    curr = curr + timedelta(minutes=5)
                    minutes += 5


        # print diagram
        print("diagram")
        # fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
        fig = plt.figure()
        fig, ax = plt.subplots()
        fig = matplotlib.pyplot.gcf()
        fig.set_size_inches(18.5, 10.5)
        # fig.savefig('test2png.png', dpi=100)
        # fig, ax = plt.subplots(figsize=(10 10))
        
        
        print("diagram2")

        for key, value in monthDistribution.items():
            monthDistribution[key] = value/60

        for key, value in hourDistribution.items():
            hourDistribution[key] = value/60


        mode = config["events"][event]["mode"]

        textstr = "Location: {}\nArea: {}\nDistribution of\n {}\nAddtional downtime\n incl.: {}\n".format(location, area, event, downtimeIncluded)
        print("textstr = ")
        print(textstr)
        
        direction = config["events"][event]["direction"]
        print("direction before if direction = ")
        print(direction)
        if direction == "above":
            if "high_threshold_field" in config["events"][event].keys():
                field = config["events"][event]["high_threshold_field"]
                if len(field) <= 10:
                    textstr += "High threshold field: \n " + config["events"][event]["high_threshold_field"] + "\n"
                else:
                    textstr += "High threshold field: \n "
                    i = 0
                    for element in field.split('&'):
                        if i == 0:
                            textstr +=  element + "\n"
                            i = i+1
                        else:
                            textstr += " &" + element + "\n"
            if "high_threshold_agg" in config["events"][event].keys():
                textstr += "High threshold aggregate:\n " + config["events"][event]["high_threshold_agg"] + "\n"
            if "high_threshold_value" in config["events"][event].keys():
                textstr += "High threshold value: " + str(config["events"][event]["high_threshold_value"]) + "\n"
        elif direction == "below":
            if "low_threshold_field" in config["events"][event].keys():
                field = config["events"][event]["low_threshold_field"]
                if len(field) <= 10:
                    textstr += "Low threshold field: \n " + config["events"][event]["low_threshold_field"] + "\n"
                else:
                    textstr += "Low threshold field: \n "
                    i = 0
                    for element in field.split('&'):
                        if i == 0:
                            textstr +=  element + "\n"
                            i = i+1
                        else:
                            textstr += " &" + element + "\n"
            if "low_threshold_agg" in config["events"][event].keys():
                textstr += "Low threshold aggregate: " + config["events"][event]["low_threshold_agg"] + "\n"
            if "low_threshold_value" in config["events"][event].keys():
                textstr += "Low threshold value: " + str(config["events"][event]["low_threshold_value"]) + "\n"
        else:
            print("unknown direction")
            quit()

        if "downtime_before" in config["events"][event].keys():
            textstr += "Downtime before: " + config["events"][event]["downtime_before"] + "\n"
        if "downtime_after" in config["events"][event].keys():
            textstr += "Downtime after: " + config["events"][event]["downtime_after"] + "\n"


        if mode == "hour":
            ax.bar(range(len(hourDistribution.keys())), list(hourDistribution.values()), align="center" )
            ax.set_xticks(range(len(hourDistribution.keys())),hourDistribution.keys() )
            ax.set_xlabel("Hours in day")
        elif mode == "month":
            ax.bar(range(len(monthDistribution.keys())), list(monthDistribution.values()), align="center" )
            ax.set_xticks(range(len(monthDistribution.keys())),monthDistribution.keys() )
            ax.set_xlabel("Months in year")
        ax.set_ylabel("Downtime in hours\nfrom {} to {}".format(config["start_date"][:10], config["end_date"][:10]))
        ax.set_title(" - ".join([config["events"][event]["name"],config["location"], config["area"]]))
        plt.text(0.02, 0.3, textstr, fontsize=14, transform=plt.gcf().transFigure)
        plt.subplots_adjust(left=0.35, bottom= 0.15)
        plt.savefig(config["location"] + '_' + config["events"][event]["name"] + '_aggregateTime.png', dpi=100)
        print(config["location"])
        print(config["events"][event])

        manager = plt.get_current_fig_manager()
        #manager.window.state('zoomed')
        plt.pause(1)

        # plt.plot()
        # plt.show()

def countEventTimes():
    # directory = "WeatherResults\Hamburg" # geht nicht da backslash
    directory = "output_pipeline" # geht

    output_df = pd.DataFrame(columns=["key", "duration"])

    for dataFileName in os.listdir(directory):
        if not dataFileName.endswith("events.parquet"):
            continue
        print(dataFileName)
        df = pd.read_parquet(os.path.join(directory, dataFileName))
        df.to_csv(dataFileName.split('.')[0] + ".csv")
        durations = []
        keys = []
        for idx, row in df.iterrows():
            if row["start_datetime"][-5] in ["+", "-"]:
                start_string = row["start_datetime"][0:len(row["start_datetime"]) - 5] + row["start_datetime"][-5] \
                + "0" + row["start_datetime"][-4:len(row["start_datetime"])]
            else:
                start_string = row["start_datetime"]
            if row["end_datetime"][-5] in ["+", "-"]:
                end_string = row["end_datetime"][0:len(row["end_datetime"]) - 5] + row["end_datetime"][-5] \
                + "0" + row["end_datetime"][-4:len(row["end_datetime"])]
            else:
                end_string = row["end_datetime"]

            # start = datetime.strptime(start_string, "%Y-%m-%dT%H:%M:%S%z")
            # end = datetime.strptime(end_string, "%Y-%m-%dT%H:%M:%S%z")
            # start = datetime.strptime(row["start_datetime"], "%Y-%m-%dT%H:%M:%S %Z%z")
            # end = datetime.strptime(row["end_datetime"], "%Y-%m-%dT%H:%M:%S %Z%z")
            start = getDate(row["start_datetime"])
            end = getDate(row["end_datetime"])

            durations.append(end-start)
            keys.append("_".join([row["event_name"],row["type"], row["location_name"],row["area_name"]]))
        df["duration"] = durations
        df["key"] = keys
        df = df.drop(['end_datetime', 'start_datetime', 'area_name', 'location_name'], axis=1)
        #df = df.groupby(["key"])["duration"].sum()
        output_df = pd.concat([output_df, df], ignore_index=True)
    result = {k: timedelta() for k in output_df["key"].unique()}
    #print(output_df.head())
    for idx, row in output_df.iterrows():
        result[row["key"]] += row["duration"]
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(result)
    output_df = output_df.groupby(["key"])[["duration"]].sum()
    #print(output_df)
    print(directory) # WeatherResults\Hamburg
    # print("Events_duration" + directory.split("/")[-1] + ".csv") # Events_durationWeatherResults\Hamburg.csv
    output_df.to_csv("Events_duration" + directory.split("/")[-1] + ".csv")    # orig geht nur ohne backslash im directory
    # output_df.to_csv("Events_duration" + "test" + ".csv")


# interner check
def countValue():
    # fileLocation = "WeatherResults\Hamburg"
    fileLocation = "results"
    precipSum = 0

    for dataFileName in os.listdir(fileLocation):
        if not dataFileName.endswith("all.parquet"):
            continue
        # print(dataFileName) # HH_vis_rain_Hamburg_all.parquet
        df = pd.read_parquet(os.path.join(fileLocation, dataFileName))
        print(df.head(10))
        # df = df[(df["field_name"] == "precip_type_5min:idx == 1 & precip_5min:mm") & (df["agg_name"] == "average")]
        df = df[(df["field_name"] == "precip_type_5min:idx == 0 & visibility:m") & (df["agg_name"] == "min")] # ????? falsche results!
        print("before fill NaN")
        print(df.head(10))

        # Achtung = bei Sichtweite muss eine hohe Dummy Zahl eingegeben werden!
        df.fillna(0, inplace=True) # falsch -> Nan != 0,0
        print("after fill NaN")
        print(df.head(10))
        precipSum += df["field_value"].sum() # 474.9435448843631 ("precip_type_5min:idx == 1 & precip_5min:mm")
        # precipSum += df["field_value"].mean() # 474.9435448843631 ("precip_type_5min:idx == 1 & precip_5min:mm") #???? zu hoher mean
        #print(df.head())

    print(precipSum)

# check ein einem punkt
def countPrecipationRaw():
    # fileLocation = "WeatherResults\Hamburg"
    fileLocation = "results"
    precipSum = 0

    for dataFileName in os.listdir(fileLocation):
        print("dataFileName = " + dataFileName)
        # dataFileName = HH_vis_rain.txt
        # dataFileName = HH_vis_rain.yml
        # dataFileName = HH_vis_rain_events.parquet
        if not dataFileName.endswith(".parquet"):
            # print("if dataFileName = " + dataFileName) # HamburgAnalysis_P0(copy)_P0.txt
            continue
        # if "2021" in dataFileName or "2022" in dataFileName: # welche txt wird benötigt?
        if "2020" in dataFileName or "2022" in dataFileName: # welche txt wird benötigt?
            # print("dataFileName")
            # print(dataFileName)
            continue
        df = pd.read_parquet(os.path.join(fileLocation, dataFileName))
        #print("df")
        print(df)
        #print(df["precip_type_5min:idx"])
        df = df[df["precip_type_5min:idx"] == 1] # error
        df = df[(df["lat"] == 53.624071) & (df["lon"] == 9.992043)] #airport nähe
        print(df.head())
        precipSum += df["precip_5min:mm"].sum()

    print(precipSum)

if __name__ == "__main__":
    aggregateDiagram_Statistic_all()
    aggregateDiagram_rain_snow() # categories
    aggregateDiagram_rain_noNaN()
    aggregateDiagram_visibility() # 
    aggregateDiagram_origJohannes() # orig
    # aggregateDiagram_all() # allg Übersicht, ohne tresholds->categories
    aggregateTime() # *.txt, unterer Teil listet events mit / ohne downtime auf -> rain geht nicht!
    countEventTimes() # Pfad directory = anpassen!
    countPrecipationRaw() # geht nicht, Input unbekannt!
    countValue() # geht -> #realistische Werte?

if = precip_type_5min


KeyboardInterrupt: 